In [1]:
%pylab inline
import pandas

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pandas.read_csv("CND-features_processed.csv")

### Drop features which has many equal values (more than 80% inside a given CND):

In [3]:
TODROP = []
for CND in unique(df.CND):
    dfg = df[df.CND==CND]
    dfg = dfg[dfg.columns[1:-1]]
    TODROP = TODROP + list(dfg.columns[(len(dfg)-dfg.nunique())/len(dfg) > 0.8])
TODROP = array(TODROP)
TODROP = unique(TODROP)
print("Dropping",len(TODROP),"features...")
df = df.drop(TODROP, axis=1)

Dropping 44 features...


### Drop one from each correlated ($C_{Pearson/Spearman}>0.9$) feature pairs:
1. Also, here the appropriate features discussed in previous step were ommited
2. The remaining, independent ($C_{Pearson/Spearman}<0.9$) variables of each CND are saved in the folder "$\textit{filter}$"

In [4]:
THRESHOLD = 0.9
def ReduceCorr(dataframe, METHOD):
    corr_matrix = dataframe.corr(method=METHOD).abs()
    upper = corr_matrix.where(triu(ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > THRESHOLD)]
    dataframe = dataframe.drop(to_drop, axis=1)
    return dataframe

In [5]:
for CND in unique(df.CND):
    try:
        what2del = loadtxt("deletefeatures/"+CND+".txt", dtype="str")
        dfnew = df[["Id"]+list(setdiff1d(array(df.columns[1:-1]), what2del))+["CND"]]
        dfnew = dfnew[dfnew.CND==CND]
    except:
        dfnew = df[df.CND==CND]
    savetxt("filter/"+CND+".txt", 
            array(ReduceCorr(ReduceCorr(dfnew, "pearson"), "spearman").columns[1:-1]), fmt="%s")